[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/falcon/falcon-40b-chatbot.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/llm-field-guide/falcon/falcon-40b-chatbot.ipynb)

In [ ]:
# pip install required packages
!pip install transformers
!pip install torch
!pip install tokenizers
!pip install jsonargparse[signatures]  # CLI
!pip install bitsandbytes # quantize
!pip install zstandard  # prepare_redpajama.py
!pip install scipy
!pip install loralib
!pip install einops
!pip install peft
!pip install accelerate
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.0/187.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.2/594.2 kB 34.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# load cover letter dataset on huggingface
from datasets import load_dataset
dataset = load_dataset("ShashiVish/cover-letter-dataset")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/813 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/349 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Job Title', 'Preferred Qualifications', 'Hiring Company', 'Applicant Name', 'Past Working Experience', 'Current Working Experience', 'Skillsets', 'Qualifications', 'Cover Letter'],
        num_rows: 813
    })
    test: Dataset({
        features: ['Job Title', 'Preferred Qualifications', 'Hiring Company', 'Applicant Name', 'Past Working Experience', 'Current Working Experience', 'Skillsets', 'Qualifications', 'Cover Letter'],
        num_rows: 349
    })
})

In [ ]:
import pandas as pd
train_df = pd.DataFrame(dataset['train'])
test_df = pd.DataFrame(dataset['test'])

In [ ]:
col_to_keep = train_df['Cover Letter']
col_to_keep2 = test_df['Cover Letter']

# Select the remaining 9 columns and concatenate them into a new column
concatenated_col = train_df.drop(columns=['Cover Letter']).apply(lambda row: ' '.join(row.astype(str)), axis=1)
concatenated_col2 = test_df.drop(columns=['Cover Letter']).apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Create a new DataFrame with the two columns
falcon_train_df = pd.DataFrame({'CoverLetter': col_to_keep, 'Context': concatenated_col})
falcon_test_df = pd.DataFrame({'CoverLetter': col_to_keep2, 'Context': concatenated_col2})

In [ ]:
from datasets import load_dataset, Dataset
import pandas as pd

def gen_prompt(text_input):
    return f"""
    <human>: {text_input["Context"]}
    <assistant>: {text_input["CoverLetter"]}
    """.strip()

def gen_and_tok_prompt(text_input):
    full_input = gen_prompt(text_input)
    tok_full_prompt = tokenizer(full_input, padding = True , truncation =True)
    return tok_full_prompt

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch import cuda, bfloat16

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Specify the destination folder in your Google Drive
destination_folder = '/content/drive/My Drive'

Mounted at /content/drive


In [ ]:
#  load the falcon 7b model and tokenizer
 model = AutoModelForCausalLM.from_pretrained(
    "tiiuae/falcon-7b-instruct",
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(
    "tiiuae/falcon-7b-instruct",
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modeling_falcon.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [ ]:
tokenizer.pad_token = tokenizer.eos_token

# converting dataset to dataframe
train_data = Dataset.from_pandas(falcon_train_df)
test_data = Dataset.from_pandas(falcon_test_df)

train_data = train_data.map(gen_and_tok_prompt)
test_data = test_data.map(gen_and_tok_prompt)

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

In [ ]:
training_args = transformers.TrainingArguments(
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=4,
    logging_steps=25,
    output_dir="output_dir",
    save_strategy='epoch',
    optim="paged_adamw_8bit",
    lr_scheduler_type = 'cosine',
    warmup_ratio = 0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
# create your own prompt
prompt = f"""
    <human>: I would like to write a cover letter for a data science position at XYZ Tech Solutions. Could you write me one please?
    <assistant>:
    """.strip()

# encode the prompt
encoding = tokenizer(prompt, return_tensors= "pt").to(model.device)

#import torch
import torch

# set the generation configuration params
gen_config = model.generation_config
gen_config.max_new_tokens = 250
gen_config.temperature = 0.2
gen_config.top_p = 0.7
gen_config.num_return_sequences = 1
gen_config.pad_token_id = tokenizer.eos_token_id
gen_config.eos_token_id = tokenizer.eos_token_id
gen_config.do_sample = True  # Enable sampling
gen_config.use_cache = False  # Disable cache

In [ ]:
# produce a prediction
with torch.inference_mode():
    outputs = model.generate(input_ids = encoding.input_ids, attention_mask = encoding.attention_mask,generation_config = gen_config )
print(tokenizer.decode(outputs[0], skip_special_tokens = True ))

In [ ]:
generated_cover_letters = []

In [ ]:
# loop through the 'Context' column in the dataset to produce a generated cover letter
for prompt in test_data['Context']:
    prompt = f"""
        <human>: {prompt}
        <assistant>:
        """.strip()

    # encode the prompt
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.inference_mode():
        outputs = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, generation_config=gen_config)

    # decode the generated cover letter
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # append it to the list
    generated_cover_letters.append(decoded)

In [ ]:
test_data = test_data.add_column("generated_cover_letters", generated_cover_letters)

In [ ]:
test_data_df = test_data.to_pandas()

In [ ]:
test_data_df.to_csv('data.csv')
!cp data.csv "/content/drive/MyDrive/"

In [ ]:
#evaluating the model
import pandas as pd
import re

df = pd.read_csv('/content/data.csv')

In [ ]:
df.head()

,Unnamed: 0,CoverLetter,Context,input_ids,attention_mask,generated_cover_letters
0,0,"Dear Hiring Manager,\n\nI am writing to expres...",Data Scientist BSc focused on data Science/co...,[ 39 15564 48190 204 4745 39814 341 36...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,<human>: Data Scientist BSc focused on data S...
1,1,"Dear Hiring Manager,\n\nI am writing to expres...",Data Scientist Experience working within comm...,[ 39 15564 48190 204 4745 39814 10737 16...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,<human>: Data Scientist Experience working wi...
2,2,"Dear Hiring Manager,\n\nI am writing to expres...",Data Scientist Excellent English written and ...,[ 39 15564 48190 204 4745 39814 19792 35...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,<human>: Data Scientist Excellent English wri...
3,3,I am thrilled to apply for the Generative AI E...,Generative AI Engineer PhD in AI Facebook Oliv...,[ 39 15564 48190 3367 998 8317 20161 128...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,<human>: Generative AI Engineer PhD in AI Face...
4,4,"Dear Hiring Manager,\n\nI am writing to expres...",Data Scientist 1. 3+ years of software develop...,[ 39 15564 48190 4745 39814 204 28 ...,[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1...,<human>: Data Scientist 1. 3+ years of softwar...


In [ ]:
# remove all text after <assistant>:
import re

def extract_text_after_assistant(text):
    match = re.search(r'<assistant>:(.*)', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return None

# Apply the function to create a new column
df['generated_cover_letter_cleaned'] = df['generated_cover_letters'].apply(extract_text_after_assistant)

In [ ]:
df.to_csv('data_cleaned.csv')

In [ ]:
!pip install evaluate
!pip install rouge_score
import evaluate
import matplotlib.pyplot as plt

In [ ]:
def calculate_evaluation_score(predictions, actuals):
    # initialize evaluation tools
    bleu = evaluate.load('bleu')
    rouge = evaluate.load('rouge')

    total_scores = {
        'bleu': 0,
        'rouge1': 0,
        'rouge2': 0,
        'rougeL': 0,
        'rougeLsum': 0,
    }

    num_samples = len(predictions)

    for i in range(num_samples):
        prediction = predictions[i]
        actual = actuals[i]

        # calculate BLEU and ROUGE scores for each pair
        bleu_results = bleu.compute(predictions=[prediction], references=[actual])
        rouge_results = rouge.compute(predictions=[prediction], references=[actual])

        # accumulate scores
        total_scores['bleu'] += bleu_results['bleu']
        total_scores['rouge1'] += rouge_results['rouge1']
        total_scores['rouge2'] += rouge_results['rouge2']
        total_scores['rougeL'] += rouge_results['rougeL']
        total_scores['rougeLsum'] += rouge_results['rougeLsum']

    # calculate average scores
    for metric in total_scores:
        total_scores[metric] /= num_samples

    return total_scores


# extract "CoverLetter" and "generated_cover_letter_cleaned" columns
predictions = df['generated_cover_letter_cleaned'].tolist()
actuals = df['CoverLetter'].tolist()

# calculate the overall evaluation score
evaluation_score = calculate_evaluation_score(predictions, actuals)

# print or use the evaluation score as needed
print("falcon-7b-finetuned: ", evaluation_score)

falcon-7b-finetuned:  {'bleu': 0.3154793473431506, 'rouge1': 0.5716218087136334, 'rouge2': 0.37996006591356507, 'rougeL': 0.4217133365390919, 'rougeLsum': 0.47746530825225103}


In [ ]:
#perform evaluation on 10 rows of data (for comparison against llama)
df_10_rows = df.head()

In [ ]:
# extract "CoverLetter" and "generated_cover_letter_cleaned" columns
predictions = df_10_rows['generated_cover_letter_cleaned'].tolist()
actuals = df_10_rows['CoverLetter'].tolist()

# calculate the overall evaluation score
evaluation_score_10_rows = calculate_evaluation_score(predictions, actuals)

# print or use the evaluation score as needed
print("falcon-7b-finetuned: ", evaluation_score_10_rows)

falcon-7b-finetuned:  {'bleu': 0.44797507572252115, 'rouge1': 0.6853324344766888, 'rouge2': 0.48842181411120106, 'rougeL': 0.5007804805989714, 'rougeLsum': 0.6092683542519853}
